In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())


cuda:1


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/MURI Aug17 Thin Plate'

In [4]:
# !pip install smt

In [5]:
#Material Properties This link - https://www.mathworks.com/help/pde/ug/nonlinear-heat-transfer-in-a-thin-plate.html#heatTransferThinPlateExample-1
k = 400
rho = 8960
cp = 386
t_z = 0.01
stef_bolt = 5.670373e-8
hc = 1
Ta = 300
emiss = 0.5


In [6]:
label = "3D_HTTP_atanh"
loss_thresh = 20000
x = np.linspace(0,1,100).reshape(-1,1)
y = np.linspace(0,1,100).reshape(-1,1)
t = np.linspace(0,1,100).reshape(-1,1) #t is actually from 0 to 5000, let us scale it to 0 to 1

X,Y,T = np.meshgrid(x,y,t)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xyt = np.hstack((X,Y,T))

initial_pts = np.logical_and(T==0,Y!=0).reshape(-1,)

DBC_pts = (Y == 0).reshape(-1,)


NBC_pts_x0 = (X == 0).reshape(-1,)
NBC_pts_x1 = (X == 1).reshape(-1,)

NBC_pts_y0 = (Y == 0).reshape(-1,)
NBC_pts_y1 = (Y == 1).reshape(-1,)

xyt_initial = xyt[initial_pts,:]
xyt_DBC = xyt[DBC_pts,:]

xyt_NBC_x0 = xyt[NBC_pts_x0,:]
xyt_NBC_x1 = xyt[NBC_pts_x1,:]

#xyt_NBC_y0 = xyt[NBC_pts_y0,:]
xyt_NBC_y1 = xyt[NBC_pts_y1,:]

u_initial = 300*np.ones((np.shape(xyt_initial)[0],1))
u_DBC = 1000*np.ones((np.shape(xyt_DBC)[0],1))

xyt_I_DBC = np.vstack((xyt_initial,xyt_DBC))
#xyt_NBC = np.vstack((xyt_NBC_1,xyt_NBC_2,xyt_NBC_3,xyt_NBC_4))
xyt_NBC_x = np.vstack((xyt_NBC_x0,xyt_NBC_x1))
#xyt_NBC_y = np.vstack((xyt_NBC_y0,xyt_NBC_y1))
xyt_NBC_y = np.vstack((xyt_NBC_y1))

u_I_DBC = np.vstack((u_initial,u_DBC))


lb_xyt = xyt[0]
ub_xyt = xyt[-1]

In [7]:
fea_data = scipy.io.loadmat('./../3D_HTTP_FEA.mat')
xy = fea_data['xy']
t = fea_data['t']/3000
xyt = np.zeros((497*101,3))
u_true = np.ones((497*101,1))


for i in range(101):
    t_temp = t[0,i]*np.ones((497,1))
    xyt[497*i:497*(i+1)] = np.hstack((xy,t_temp))
    u_true[497*i:497*(i+1)] = fea_data['u'][:,i].reshape(-1,1)
    #print(i)
#print(xyt)

xyt_test_tensor = torch.from_numpy(xyt).float().to(device)
u_true_norm = np.linalg.norm(u_true,2)

In [8]:
def trainingdata(N_D,N_N,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    #choose random N_u points for training
    idx = np.random.choice(xyt_I_DBC.shape[0], N_D, replace=False) 
    xyt_D = xyt_I_DBC[idx,:] #choose indices from  set 'idx' (x,t)
    u_D = u_I_DBC[idx].reshape(-1,1)      #choose corresponding u

    idx = np.random.choice(xyt_NBC_x.shape[0], N_D, replace=False) 
    xyt_Nx = xyt_NBC_x[idx,:] #choose indices from  set 'idx' (x,t)

    idx = np.random.choice(xyt_NBC_y.shape[0], N_D, replace=False) 
    xyt_Ny = xyt_NBC_y[idx,:] #choose indices from  set 'idx' (x,t)

    '''Collocation Points'''
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xyt_coll = lb_xyt + (ub_xyt - lb_xyt)*samples
    xyt_coll = np.vstack((xyt_coll, xyt_D,xyt_Nx,xyt_Ny)) # append training points to collocation points 

    return xyt_coll, xyt_D, u_D, xyt_Nx,xyt_Ny

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
        
        self.alpha = Parameter(torch.ones((50,len(layers)-2)))
        self.alpha.requiresGrad = True
        
        self.n = torch.tensor(n_val)

    
            
    'foward pass'
    def forward(self,xyt):
        if torch.is_tensor(xyt) != True:         
            xyt = torch.from_numpy(xyt)                
        
        ubxyt = torch.from_numpy(ub_xyt).float().to(device)
        lbxyt = torch.from_numpy(lb_xyt).float().to(device)
    
                      
        #preprocessing input 
        xyt = (xyt - lbxyt)/(ubxyt - lbxyt)
        
        #convert to float
        a = xyt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.n*self.alpha[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_D(self,xyt_D,u_D):
                
        loss_bc = self.loss_function(self.forward(xyt_D), u_D)
                
        return loss_bc
    
    def loss_N(self,xyt_Nx,xyt_Ny,N_hat):
        
        g1 = xyt_Nx.clone()             
        g1.requires_grad = True
        u1 = self.forward(g1)
        
        u1_x_y_t = autograd.grad(u1,g1,torch.ones([xyt_Nx.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du1_dx = u1_x_y_t[:,[0]]
        
        g2 = xyt_Ny.clone()             
        g2.requires_grad = True
        u2 = self.forward(g2)
        
        u2_x_y_t = autograd.grad(u2,g2,torch.ones([xyt_Ny.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du2_dy = u2_x_y_t[:,[1]]
               
        loss_N1 = self.loss_function(du1_dx,N_hat)
        loss_N2 = self.loss_function(du2_dy,N_hat)
        
        #return loss_N1+loss_N2       
        return loss_N1 + loss_N2
    
    def loss_PDE(self, xyt_coll, f_hat):
        
        g = xyt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_y_t = autograd.grad(u,g,torch.ones([xyt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy_tt = autograd.grad(u_x_y_t,g,torch.ones(xyt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dt = u_x_y_t[:,[2]]
        
        d2u_dx2 = u_xx_yy_tt[:,[0]]
        d2u_dy2 = u_xx_yy_tt[:,[1]]    
        

        f = rho*cp*t_z*du_dt/3000 - k*t_z*(d2u_dx2+d2u_dy2) + 2*hc*(u-Ta) + 2*emiss*stef_bolt*(torch.pow(u,4)-Ta**4) 
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat):

        loss_D = self.loss_D(xyt_D,u_D)
        loss_N = self.loss_N(xyt_Nx,xyt_Ny,N_hat)
        loss_f = self.loss_PDE(xyt_coll,f_hat)
        
        loss_val = loss_D + loss_N + loss_f
        
        #print(self.iter,"loss_D:",loss_D.cpu().detach().numpy(),"loss_N:",loss_N.cpu().detach().numpy(),"loss_f:",loss_f.cpu().detach().numpy())
        
        return loss_val
       
    'test neural network'
    def test(self):
        u_pred = self.forward(xyt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
        
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [10]:
def train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xyt_coll_np_array, xyt_D_np_array, u_D_np_array,xyt_Nx_np_array,xyt_Ny_np_array = trainingdata(N_D,N_N,N_f,(reps)*22)

    xyt_coll = torch.from_numpy(xyt_coll_np_array).float().to(device)
    xyt_D = torch.from_numpy(xyt_D_np_array).float().to(device)
    u_D = torch.from_numpy(u_D_np_array).float().to(device)
    xyt_Nx = torch.from_numpy(xyt_Nx_np_array).float().to(device)
    xyt_Ny = torch.from_numpy(xyt_Ny_np_array).float().to(device)

    N_hat = torch.zeros(xyt_Nx.shape[0],1).to(device)    
    f_hat = torch.zeros(xyt_coll.shape[0],1).to(device)

    
    for i in range(max_iter):
        train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,i)

        loss_np = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])

    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [13]:

max_reps = 10
max_iter = 100

train_loss_full = []
test_mse_full = []
test_re_full = []
alpha_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

n_val = 1.0 


for reps in range(max_reps):
    print(label)
    train_loss = []
    test_mse_loss = []
    test_re_loss = []
    alpha_val = []


    print(reps)

    torch.manual_seed(reps*36)
    N_D = 5000 #Total number of data points for 'y'
    N_N = 3500
    N_f = 10000 #Total number of collocation points 

    layers = np.array([3,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

    PINN = Sequentialmodel(layers,n_val)

    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())


    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.1, 
                              max_iter = 20, 
                              max_eval = 30, 
                              tolerance_grad = 1e-8, 
                              tolerance_change = 1e-8, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')



    nan_flag = train_model(max_iter,reps)


    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time
    alpha_full.append(alpha_val)



    #print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "alpha": alpha_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

3D_HTTP_atanh
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 894870.9 Test MSE 325713.2632569917 Test RE 1.0117124258788912
1 Train Loss 528936.56 Test MSE 279647.29161598574 Test RE 0.9374425693316915
2 Train Loss 403515.34 Test MSE 191058.0671917834 Test RE 0.7748578767582727
3 Train Loss 244788.72 Test MSE 86939.00396886282 Tes

96 Train Loss 141687.5 Test MSE 34830.93812035957 Test RE 0.3308429983326183
97 Train Loss 141633.4 Test MSE 34826.82821748274 Test RE 0.33082347872193535
98 Train Loss 141588.86 Test MSE 34921.45484851346 Test RE 0.3312726077051339
99 Train Loss 141508.8 Test MSE 34969.00208346655 Test RE 0.3314980527611742
Training time: 210.94
3D_HTTP_atanh
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_

92 Train Loss 136124.58 Test MSE 33725.78580308682 Test RE 0.3255520248675977
93 Train Loss 135543.61 Test MSE 33560.13447764036 Test RE 0.32475153191972245
94 Train Loss 135377.52 Test MSE 33535.554940623566 Test RE 0.3246325856682547
95 Train Loss 135271.42 Test MSE 33472.85578907648 Test RE 0.3243289719314779
96 Train Loss 135154.78 Test MSE 33424.303676718046 Test RE 0.3240936682707542
97 Train Loss 134991.06 Test MSE 33260.61201052097 Test RE 0.3232990885850895
98 Train Loss 134789.12 Test MSE 33108.88386148536 Test RE 0.3225608333276164
99 Train Loss 134671.88 Test MSE 32873.637577386646 Test RE 0.32141285583640694
Training time: 392.64
3D_HTTP_atanh
2
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)


88 Train Loss 161759.3 Test MSE 40373.90565270976 Test RE 0.3561965790271399
89 Train Loss 161412.33 Test MSE 40237.60877599025 Test RE 0.35559483486805216
90 Train Loss 161163.02 Test MSE 40090.9977428911 Test RE 0.35494641534466126
91 Train Loss 161032.95 Test MSE 39944.841664876105 Test RE 0.3542988267760457
92 Train Loss 160695.73 Test MSE 39731.88070247088 Test RE 0.3533531145081548
93 Train Loss 160406.97 Test MSE 39604.153515003985 Test RE 0.35278469024105996
94 Train Loss 160289.97 Test MSE 39638.88787613321 Test RE 0.35293935918572356
95 Train Loss 160116.69 Test MSE 39432.69654156005 Test RE 0.35202021229473845
96 Train Loss 159822.81 Test MSE 39206.499194806136 Test RE 0.35100911537260854
97 Train Loss 159558.77 Test MSE 39297.79682673899 Test RE 0.3514175638101432
98 Train Loss 159328.34 Test MSE 39242.27255202035 Test RE 0.3511692152505034
99 Train Loss 159121.52 Test MSE 39229.04737477875 Test RE 0.35111003587360595
Training time: 400.37
3D_HTTP_atanh
3
Sequentialmodel(
 

84 Train Loss 125243.28 Test MSE 29662.142762256484 Test RE 0.3053097141376134
85 Train Loss 125086.7 Test MSE 29583.613446746283 Test RE 0.3049052987719247
86 Train Loss 124769.42 Test MSE 29414.0865040884 Test RE 0.30403042378108824
87 Train Loss 124522.25 Test MSE 29142.77750050885 Test RE 0.30262502092738824
88 Train Loss 124230.42 Test MSE 28950.005853016704 Test RE 0.30162246864815057
89 Train Loss 124051.984 Test MSE 28817.969832883507 Test RE 0.30093385837327397
90 Train Loss 123950.79 Test MSE 28777.824131390873 Test RE 0.30072417303862564
91 Train Loss 123685.65 Test MSE 28920.205375316054 Test RE 0.3014671870218589
92 Train Loss 123555.375 Test MSE 28930.577022189806 Test RE 0.30152123973145345
93 Train Loss 123426.79 Test MSE 28786.245731456558 Test RE 0.3007681720818861
94 Train Loss 123325.62 Test MSE 28800.201007072064 Test RE 0.30084106790508414
95 Train Loss 123215.07 Test MSE 28732.43407193053 Test RE 0.3004869196133591
96 Train Loss 123138.28 Test MSE 28724.387746749

...

75 Train Loss 96080.98 Test MSE 22760.064976440324 Test RE 0.2674398444496468
76 Train Loss 95453.11 Test MSE 22468.246563889224 Test RE 0.26571982195186694
77 Train Loss 94606.39 Test MSE 22115.263941933932 Test RE 0.263624291572978
78 Train Loss 93852.84 Test MSE 21631.899454363447 Test RE 0.2607274097219589
79 Train Loss 93615.14 Test MSE 21594.53787962715 Test RE 0.2605021545059758
80 Train Loss 93188.89 Test MSE 21538.314350134624 Test RE 0.2601628118354561
81 Train Loss 92614.03 Test MSE 21385.47609665906 Test RE 0.25923809644488716
82 Train Loss 92015.2 Test MSE 21205.70761362203 Test RE 0.2581462060892158
83 Train Loss 91724.86 Test MSE 21183.936982991578 Test RE 0.25801366043274415
84 Train Loss 91512.31 Test MSE 21001.1308311407 Test RE 0.2568979877110249
85 Train Loss 91416.54 Test MSE 21001.363274196723 Test RE 0.2568994093960904
86 Train Loss 91026.55 Test MSE 20750.692355912644 Test RE 0.2553616395454036
87 Train Loss 90612.4 Test MSE 20867.50597109487 Test RE 0.256079395

71 Train Loss 123308.69 Test MSE 29974.926752602292 Test RE 0.30691522122191833
72 Train Loss 123153.96 Test MSE 30023.4633875559 Test RE 0.30716360559237016
73 Train Loss 122972.695 Test MSE 30041.174407031765 Test RE 0.3072541910532902
74 Train Loss 122516.59 Test MSE 29877.518196663274 Test RE 0.3064161291567254
75 Train Loss 122187.67 Test MSE 29607.374675064275 Test RE 0.3050277224619151
76 Train Loss 121753.77 Test MSE 29465.88452161223 Test RE 0.3042980038435608
77 Train Loss 121372.18 Test MSE 29480.8711824394 Test RE 0.3043753785996068
78 Train Loss 120691.84 Test MSE 29164.064406723337 Test RE 0.3027355247136412
79 Train Loss 120344.37 Test MSE 29085.13920433138 Test RE 0.30232560842851974
80 Train Loss 120075.44 Test MSE 28827.31535026519 Test RE 0.3009826500585867
81 Train Loss 119711.92 Test MSE 28644.710928508513 Test RE 0.30002785987004554
82 Train Loss 119338.5 Test MSE 28637.025000207475 Test RE 0.29998760553947545
83 Train Loss 118987.47 Test MSE 28619.954801029624 Te

66 Train Loss 136701.88 Test MSE 33597.9043591981 Test RE 0.3249342245387787
67 Train Loss 136469.28 Test MSE 33546.403151562365 Test RE 0.3246850881244887
68 Train Loss 136372.31 Test MSE 33512.46675385319 Test RE 0.32452081670727567
69 Train Loss 135954.95 Test MSE 33488.05447322201 Test RE 0.32440259597783766
70 Train Loss 135511.45 Test MSE 33529.25254511152 Test RE 0.3246020798500421
71 Train Loss 135310.75 Test MSE 33476.70005341805 Test RE 0.32434759553672976
72 Train Loss 134896.06 Test MSE 33031.7802589382 Test RE 0.32218502635576707
73 Train Loss 134795.48 Test MSE 33062.17352685573 Test RE 0.3223332170416229
74 Train Loss 134627.52 Test MSE 33104.36394097425 Test RE 0.32253881508177645
75 Train Loss 134569.44 Test MSE 33114.19060298531 Test RE 0.3225866825586385
76 Train Loss 134517.88 Test MSE 33018.11000174824 Test RE 0.3221183510521663
77 Train Loss 134471.75 Test MSE 32967.349096873295 Test RE 0.32187064898526985
78 Train Loss 134180.83 Test MSE 32701.875897989456 Test R

62 Train Loss 172724.52 Test MSE 45669.24885178359 Test RE 0.37883604718663266
63 Train Loss 172372.34 Test MSE 45502.904866747594 Test RE 0.3781454885845056
64 Train Loss 171881.69 Test MSE 45406.70314011401 Test RE 0.3777455416250753
65 Train Loss 171462.66 Test MSE 45595.57154688594 Test RE 0.37853033945042425
66 Train Loss 170683.78 Test MSE 45444.67061162429 Test RE 0.377903437324105
67 Train Loss 169784.78 Test MSE 45944.45529975639 Test RE 0.37997578048760233
68 Train Loss 169050.45 Test MSE 46158.14649723204 Test RE 0.38085840369001556
69 Train Loss 168325.69 Test MSE 45797.790315714934 Test RE 0.3793688118390489
70 Train Loss 167914.52 Test MSE 45399.80069320115 Test RE 0.37771682926033756
71 Train Loss 167610.56 Test MSE 44974.58449878061 Test RE 0.3759438131386811
72 Train Loss 167468.3 Test MSE 44812.04856500894 Test RE 0.3752638770544081
73 Train Loss 167175.53 Test MSE 44587.02712984392 Test RE 0.37432050700429986
74 Train Loss 166951.34 Test MSE 44620.4478091511 Test RE 

58 Train Loss 149778.03 Test MSE 37819.73051523506 Test RE 0.34474547712760145
59 Train Loss 149031.81 Test MSE 37577.743503460326 Test RE 0.34364079180945994
60 Train Loss 148016.27 Test MSE 37299.014623725845 Test RE 0.3423639603650224
61 Train Loss 147525.1 Test MSE 37017.25660796349 Test RE 0.3410683945822131
62 Train Loss 146950.75 Test MSE 36677.99576281351 Test RE 0.3395018617579047
63 Train Loss 145814.94 Test MSE 36901.952511381875 Test RE 0.34053678772038226
64 Train Loss 145180.28 Test MSE 36843.103794259456 Test RE 0.3402651469565427
65 Train Loss 144816.42 Test MSE 36974.4923987756 Test RE 0.3408713279107732
66 Train Loss 144257.61 Test MSE 36598.6483344485 Test RE 0.3391344318548181
67 Train Loss 143720.83 Test MSE 36295.95580379916 Test RE 0.33772909863043726
68 Train Loss 143061.84 Test MSE 35830.436740580844 Test RE 0.33555631244944467
69 Train Loss 142676.62 Test MSE 35687.57155386178 Test RE 0.3348866695186494
70 Train Loss 142380.44 Test MSE 35339.37726386891 Test R